<a href="https://colab.research.google.com/github/xabhaytiwari/Biharmonic_Problem_Solution_Using_PINN/blob/main/Biharmonic_Problem_Solution_Using_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np

class NN(torch.nn.Module):
  def __init__(self) -> None:
    super(NN, self).__init__()
    self.L1 = torch.nn.Linear(2, 60) # Wider
    self.L2 = torch.nn.Linear(60, 60)
    self.L3 = torch.nn.Linear(60, 60)
    self.L4 = torch.nn.Linear(60, 60)
    self.L5 = torch.nn.Linear(60, 60) # Deeper
    self.L6 = torch.nn.Linear(60, 1)

  def forward(self, x, y):
    inputs = torch.cat([x, y], axis=1)
    x1 = torch.tanh(self.L1(inputs))
    x2 = torch.tanh(self.L2(x1))
    x3 = torch.tanh(self.L3(x2))
    x4 = torch.tanh(self.L4(x3))
    x5 = torch.tanh(self.L5(x4))
    x6 = self.L6(x5)
    return x6

  def init_weights(m):
    if isinstance(m, torch.nn.Linear):
      torch.nn.init.xavier_uniform_(m.weight)
      m.bias.data.fill_(0.0)

We'll gather Inputs, Collocation points, PDE Data and Balancing Parameter alpha.


In [2]:
from sympy import symbols, diff, sin, pi, lambdify

x1, x2 = symbols('x1 x2')
u = (1 / (2 * (pi ** 2))) * sin(pi * x1) * sin(pi * x2)
# u = (x1 ** 2) * (x2 ** 2) * ((1 - x1) ** 2) * ((1 - x2) ** 2)

laplacian_u = diff(u, x1, 2) + diff(u, x2, 2)
bi_laplacian_u = diff(laplacian_u, x1, 2) + diff(laplacian_u, x2, 2)

f_sym = bi_laplacian_u

u_x_sym = diff(u, x1)
u_y_sym = diff(u, x2)

u_xx_sym = diff(u, x1, 2)
u_yy_sym = diff(u, x2, 2)
u_xy_sym = diff(u, x1, x2)

ldu = lambdify((x1, x2), u, 'numpy')
ldu_x = lambdify((x1, x2), u_x_sym, 'numpy')
ldu_y = lambdify((x1, x2), u_y_sym, 'numpy')
ldf = lambdify((x1, x2), f_sym, 'numpy')

ldu_xx = lambdify((x1, x2), u_xx_sym, 'numpy')
ldu_yy = lambdify((x1, x2), u_yy_sym, 'numpy')
ldu_xy = lambdify((x1, x2), u_xy_sym, 'numpy')

def from_seq_to_array(items):
    out = []
    for item in items:
        out.append(np.array(item).reshape(-1, 1))
    if len(out) == 1:
        return out[0]
    return out


def data_gen_interior(collocations):
    u_gt = [ldu(x, y) for x, y in collocations]
    f_gt = [ldf(x, y) for x, y in collocations]
    return from_seq_to_array([u_gt, f_gt])

def data_gen_bdry(collocations, normal_vec):
    ybdry_vals_g1       = []
    ybdry_vals_g2       = []

    for (x, y), (n1, n2) in zip(collocations, normal_vec):
        ybdry_vals_g1.append(ldu(x, y))
        ybdry_vals_g2.append(ldu_x(x, y) * n1 + ldu_y(x, y) * n2)

    return from_seq_to_array([ybdry_vals_g1, ybdry_vals_g2])

In [3]:
import pickle as pkl
from scipy.stats import uniform
import os

N = 20000
dataname = '20000pts'

domain_data_x = uniform.rvs(size=N)
domain_data_y = uniform.rvs(size=N)

domain_data = np.array([domain_data_x,domain_data_y]).T
print(domain_data.shape)


Nb = 10000

def generate_random_bdry(Nb):
    '''
    Generate random boundary points.
    '''
    bdry_col = uniform.rvs(size=Nb*2).reshape([Nb,2])
    for i in range(Nb):
        randind = np.random.randint(0,2)
        if bdry_col[i,randind] <= 0.5:
            bdry_col[i,randind] = 0.0
        else:
            bdry_col[i,randind] = 1.0

    return bdry_col


def compute_normals(bdry_col, eps=1e-8):
    """
    Given bdry_col of shape (Nb,2) with points on the edges of [0,1]^2,
    returns two arrays n1,n2 of shape (Nb,1) giving the outward unit normals.

    Assumes:
      - if x ≈ 0 → normal = (-1, 0)
      - if x ≈ 1 → normal = ( 1, 0)
      - if y ≈ 0 → normal = ( 0,-1)
      - if y ≈ 1 → normal = ( 0, 1)
    """
    x = bdry_col[:, 0]
    y = bdry_col[:, 1]

    n1 = np.zeros_like(x)
    n2 = np.zeros_like(y)

    # left edge x=0
    mask = np.isclose(x, 0.0, atol=eps)
    n1[mask] = -1.0;  n2[mask] =  0.0

    # right edge x=1
    mask = np.isclose(x, 1.0, atol=eps)
    n1[mask] =  1.0;  n2[mask] =  0.0

    # bottom edge y=0
    mask = np.isclose(y, 0.0, atol=eps)
    n1[mask] =  0.0;  n2[mask] = -1.0

    # top edge y=1
    mask = np.isclose(y, 1.0, atol=eps)
    n1[mask] =  0.0;  n2[mask] =  1.0

    # reshape to column vectors
    return n1.reshape(-1,1), n2.reshape(-1,1)

bdry_col = generate_random_bdry(Nb)
n1_np, n2_np = compute_normals(bdry_col)
normal_vec = np.hstack([n1_np, n2_np])

print(normal_vec.shape)
print(bdry_col.shape)

if not os.path.exists('dataset/'):
    os.makedirs('dataset/')
with open('dataset/'+dataname,'wb') as pfile:
    pkl.dump(domain_data,pfile)
    pkl.dump(bdry_col,pfile)
    pkl.dump(normal_vec,pfile)


ugt,fgt = data_gen_interior(domain_data)
dirichlet_data, clamped_data = data_gen_bdry(bdry_col,normal_vec)


with open("dataset/gt_on_{}".format(dataname),'wb') as pfile:
    pkl.dump(ugt,pfile)
    pkl.dump(fgt,pfile)
    pkl.dump(dirichlet_data,pfile)
    pkl.dump(clamped_data,pfile)

(20000, 2)
(10000, 2)
(10000, 2)


In [4]:
mse_loss = torch.nn.MSELoss()

def pde(x1, x2, net):
    """
    Compute Δ²u = (u_xx + u_yy)_xx + (u_xx + u_yy)_yy
    at the points (x1, x2) via automatic differentiation.
    """
    # 1) forward pass
    u = net(x1, x2)

    # 2) first derivatives
    # u_x, u_y = torch.autograd.grad(
    #     u.sum(), (x1, x2), create_graph=True
    # )
    u_x = torch.autograd.grad(u.sum(), x1, create_graph=True)[0]
    u_y = torch.autograd.grad(u.sum(), x2, create_graph=True)[0]

    # 3) second derivatives
    u_xx = torch.autograd.grad(u_x.sum(), x1, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(), x2, create_graph=True)[0]

    # 4) Laplacian
    lap = u_xx + u_yy

    # 5) third derivatives of lap
    # lap_x, lap_y = torch.autograd.grad(
    #     lap.sum(), (x1, x2), create_graph=True
    # )
    lap_x = torch.autograd.grad(lap.sum(), x1, create_graph=True)[0]
    lap_y = torch.autograd.grad(lap.sum(), x2, create_graph=True)[0]

    # 6) fourth derivatives (bi‐Laplacian pieces)
    lap_xx = torch.autograd.grad(lap_x.sum(), x1, create_graph=True)[0]
    lap_yy = torch.autograd.grad(lap_y.sum(), x2, create_graph=True)[0]



    return lap_xx +  lap_yy



def bdry(x1,x2,n1,n2,net):
    out = net(x1,x2)
    u_x = torch.autograd.grad(out.sum(),x1,create_graph=True)[0]
    u_y = torch.autograd.grad(out.sum(),x2,create_graph=True)[0]

    return out, u_x*n1 + u_y*n2


def pdeloss(net,intx1,intx2,pdedata,bdx1,bdx2,nx1,nx2,bdrydata_diri,bdrydat_clamped,bw_diri,bw_clamped):
    bdx1  = bdx1.requires_grad_(True)
    bdx2  = bdx2.requires_grad_(True)
    pout = pde(intx1,intx2,net)
    bout_diri, bout_clamped = bdry(bdx1,bdx2,nx1,nx2,net)
    pres = mse_loss(pout,pdedata)
    bres = bw_diri*mse_loss(bout_diri,bdrydata_diri) + bw_clamped*mse_loss(bout_clamped,bdrydat_clamped)
    diri_loss = mse_loss(bout_diri,bdrydata_diri)
    clamped_loss = mse_loss(bout_clamped,bdrydat_clamped)


    loss = pres + bres

    return loss, pres, bres, diri_loss, clamped_loss

In [5]:
from torch.autograd import Variable

def from_numpy_to_tensor(numpys,require_grads,dtype=torch.float32):
    outputs = list()
    for ind in range(len(numpys)):
        outputs.append(
            Variable(torch.from_numpy(numpys[ind]),requires_grad=require_grads[ind]).type(dtype)
        )

    return outputs

In [7]:
from time import time
from tracemalloc import start
import numpy as np
import torch
from torch.utils.data import TensorDataset
import torch.optim as opt
import matplotlib.pyplot as plt
import torch.utils.data.dataset as Dataset
import torch.utils.data.dataloader as Dataloader
from torch.autograd import Variable
import pickle as pkl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from time import time
from torch.utils.data import TensorDataset

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
torch.set_default_dtype(torch.float32)
start_time = time()

y = NN()
y.apply(NN.init_weights)
y.to(device)

dataname = '20000pts'
name = 'results/'
bw_dir = 1e4   # Start with 10,000
bw_clamp = 1e4 # Start with 10,000

if not os.path.exists(name):
    os.makedirs(name)
if not os.path.exists(name+"y_plot/"):
    os.makedirs(name+"y_plot/")

params = list(y.parameters())

# --- 1. Load all data from files ---
with open("dataset/"+dataname,'rb') as pfile:
    int_col = pkl.load(pfile)
    bdry_col = pkl.load(pfile)
    normal_vec = pkl.load(pfile)

with open("dataset/gt_on_{}".format(dataname),'rb') as pfile:
    u_gt_np = pkl.load(pfile)
    f_np = pkl.load(pfile)
    dirichlet_data_np = pkl.load(pfile)
    clamped_data_np = pkl.load(pfile)

print(f"Interior points: {int_col.shape}")
print(f"Boundary points: {bdry_col.shape}")
print(f"Forcing term: {f_np.shape}")
print(f"Dirichlet data: {dirichlet_data_np.shape}")
print(f"Clamped data: {clamped_data_np.shape}")

def compute_derivatives(net, x, y):
    """ Computes u, 1st, and 2nd derivatives """
    u = net(x, y)

    # 1st derivatives
    u_x, u_y = torch.autograd.grad(
        u.sum(), (x, y), create_graph=True
    )

    # 2nd derivatives
    u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0]
    u_xy = torch.autograd.grad(u_x.sum(), y, create_graph=True)[0]

    return u, u_x, u_y, u_xx, u_yy, u_xy

# --- 2. Create Tensors and DataLoaders ---

# Create a TensorDataset for INTERIOR points AND their forcing term ---
intx1_np, intx2_np = np.split(int_col, 2, axis=1)

# Convert interior data to tensors (no grad needed here, will be set in loop)
tintx1 = torch.from_numpy(intx1_np).float()
tintx2 = torch.from_numpy(intx2_np).float()
f_tensor = torch.from_numpy(f_np).float()

# Create a dataset and loader for interior points
interior_dataset = TensorDataset(tintx1, tintx2, f_tensor)
loader = torch.utils.data.DataLoader(interior_dataset, batch_size=2000, shuffle=True)


# Create Tensors for BOUNDARY data (with requires_grad=True) ---
bdx1_np, bdx2_np = np.split(bdry_col, 2, axis=1)
nx1_np, nx2_np = np.split(normal_vec, 2, axis=1)

# Boundary coordinates MUST have requires_grad=True
# Normals and ground truth data do NOT need gradients.
[tbdx1, tbdx2] = from_numpy_to_tensor([bdx1_np, bdx2_np], [True, True], dtype=torch.float32)
[tnx1, tnx2] = from_numpy_to_tensor([nx1_np, nx2_np], [False, False], dtype=torch.float32)
[bdrydata_dirichlet, bdrydata_clamped] = from_numpy_to_tensor(
    [dirichlet_data_np, clamped_data_np], [False, False], dtype=torch.float32
)

# Move all *full* boundary tensors to the device once
tbdx1 = tbdx1.to(device)
tbdx2 = tbdx2.to(device)
tnx1 = tnx1.to(device)
tnx2 = tnx2.to(device)
bdrydata_dirichlet = bdrydata_dirichlet.to(device)
bdrydata_clamped = bdrydata_clamped.to(device)

# Note: We won't use ygt for training, only for final error calculation
ygt = torch.from_numpy(u_gt_np).float().to(device)

optimizer = opt.Adam(params, lr=1e-3)
scheduler = opt.lr_scheduler.ReduceLROnPlateau(optimizer, patience=400)
train_start_time = time()
# --- 3. Define the Closure ---
def closure():
    tot_loss = 0
    tot_pres = 0
    tot_diri_loss = 0
    tot_clamped_loss = 0

    # Iterate over the DataLoader
    for i, (intx1_batch, intx2_batch, f_batch) in enumerate(loader):
       optimizer.zero_grad()

       # Set requires_grad=True for the BATCH coordinates
       ttintx1 = intx1_batch.to(device).requires_grad_(True)
       ttintx2 = intx2_batch.to(device).requires_grad_(True)

       # This is the ground-truth forcing term for the batch
       f_gt_batch = f_batch.to(device)

       # --- Pass the correct batch data to the loss function ---
       # We use the *batched* interior points and *batched* forcing term
       # We use the *full* set of boundary points
       loss, pres, bres, diri_loss, clamped_loss = pdeloss(
           y,
           ttintx1, ttintx2, f_gt_batch,  # Batched interior data
           tbdx1, tbdx2, tnx1, tnx2,     # Full boundary data
           bdrydata_dirichlet, bdrydata_clamped,
           bw_dir, bw_clamp
       )

       loss.backward()
       optimizer.step()

       # Detach losses to free memory
       tot_loss += loss.detach()
       tot_pres += pres.detach()
       tot_diri_loss += diri_loss.detach()
       tot_clamped_loss += clamped_loss.detach()

    # Get the average loss per batch
    num_batches = len(loader)
    avg_loss = tot_loss / num_batches
    avg_pres = tot_pres / num_batches
    avg_diri = tot_diri_loss / num_batches
    avg_clamp = tot_clamped_loss / num_batches

    # scheduler.step() uses the average loss
    scheduler.step(avg_loss)

    return (
        avg_loss.cpu().numpy(),
        avg_pres.cpu().numpy(),
        avg_diri.cpu().numpy(),
        avg_clamp.cpu().numpy()
    )

# --- 4. Run the Training Loop ---
losslist = list()
print("Starting training...")
for epoch in range(12000):
    loss, presloss, diri_loss, clamped_loss = closure()
    losslist.append(loss)
    if epoch % 100 == 0:
         print(f"Epoch {epoch} | Total loss: {loss:.8f} | Loss_int: {presloss:.8f} | Loss_Clamped: {clamped_loss:.8f} | Loss_dirichlet: {diri_loss:.8f}")
train_end_time = time()
print("Training finished.")
with open("results/loss.pkl",'wb') as pfile:
    pkl.dump(losslist,pfile)

# --- (After training is finished) ---

print("Calculating final errors...")

# Use the same grid as your plots
x_pts = np.linspace(0,1,200)
y_pts = np.linspace(0,1,200)
ms_x, ms_y = np.meshgrid(x_pts,y_pts)
x_flat = np.ravel(ms_x).reshape(-1,1)
y_flat = np.ravel(ms_y).reshape(-1,1)
collocations = np.concatenate([x_flat, y_flat], axis=1)

# --- 1. Get Ground Truth (GT) values ---
u_gt_np = ldu(x_flat, y_flat).flatten()
ux_gt_np = ldu_x(x_flat, y_flat).flatten()
uy_gt_np = ldu_y(x_flat, y_flat).flatten()
uxx_gt_np = ldu_xx(x_flat, y_flat).flatten()
uyy_gt_np = ldu_yy(x_flat, y_flat).flatten()
uxy_gt_np = ldu_xy(x_flat, y_flat).flatten()

# --- 2. Get Network Prediction (NN) values ---
pt_x = torch.from_numpy(x_flat).float().to(device).requires_grad_(True)
pt_y = torch.from_numpy(y_flat).float().to(device).requires_grad_(True)

u_nn, ux_nn, uy_nn, uxx_nn, uyy_nn, uxy_nn = compute_derivatives(y, pt_x, pt_y)

u_nn_np = u_nn.data.cpu().numpy().flatten()
ux_nn_np = ux_nn.data.cpu().numpy().flatten()
uy_nn_np = uy_nn.data.cpu().numpy().flatten()
uxx_nn_np = uxx_nn.data.cpu().numpy().flatten()
uyy_nn_np = uyy_nn.data.cpu().numpy().flatten()
uxy_nn_np = uxy_nn.data.cpu().numpy().flatten()

# --- 3. Compute Errors ---
# L2 Error
l2_error_u = np.sqrt(np.mean((u_gt_np - u_nn_np)**2))
l2_error = l2_error_u / np.sqrt(np.mean(u_gt_np**2))

# H1 Error
l2_error_ux = np.sqrt(np.mean((ux_gt_np - ux_nn_np)**2))
l2_error_uy = np.sqrt(np.mean((uy_gt_np - uy_nn_np)**2))
h1_error = np.sqrt(l2_error_u**2 + l2_error_ux**2 + l2_error_uy**2) / \
           np.sqrt(np.mean(u_gt_np**2) + np.mean(ux_gt_np**2) + np.mean(uy_gt_np**2))

# H2 Error
l2_error_uxx = np.sqrt(np.mean((uxx_gt_np - uxx_nn_np)**2))
l2_error_uyy = np.sqrt(np.mean((uyy_gt_np - uyy_nn_np)**2))
l2_error_uxy = np.sqrt(np.mean((uxy_gt_np - uxy_nn_np)**2))
h2_error = np.sqrt(l2_error_u**2 + l2_error_ux**2 + l2_error_uy**2 + \
                   l2_error_uxx**2 + l2_error_uyy**2 + l2_error_uxy**2) / \
           np.sqrt(np.mean(u_gt_np**2) + np.mean(ux_gt_np**2) + np.mean(uy_gt_np**2) + \
                   np.mean(uxx_gt_np**2) + np.mean(uyy_gt_np**2) + np.mean(uxy_gt_np**2))

print(f"Relative L2 Error: {l2_error:.4e}")
print(f"Relative H1 Error: {h1_error:.4e}")
print(f"Relative H2 Error: {h2_error:.4e}")

# --- 4. Reshape for Plotting ---
ms_ysol = u_nn_np.reshape(ms_x.shape)
ms_ugt = u_gt_np.reshape(ms_x.shape)

# --- Plot Loss History ---
print("Plotting loss history...")
fig_loss = plt.figure(figsize=(10, 6))
plt.plot(losslist)
plt.yscale('log') # Log scale is essential for loss plots
plt.title('Training Loss Dynamics')
plt.xlabel('Epoch')
plt.ylabel('Total Loss')
plt.grid(True, which="both", ls="--")
plt.savefig(name + 'loss_history.png')
plt.close(fig_loss)

end_time = time()

print("\n---Training Summary ---")
print(f"Device: {device}")
print("\n[Network Architecture]")
print(y)
print("\n[Hyperparameters]")
print(f"  Optimizer: Adam (lr={1e-3})")
print(f"  Epochs (Adam): {12000}")
print(f"  Batch Size: {2000}")
print(f"  Interior Points: {int_col.shape[0]}")
print(f"  Boundary Points: {bdry_col.shape[0]}")
print(f"  Loss Weight (Dirichlet): {bw_dir:.1e}")
print(f"  Loss Weight (Clamped): {bw_clamp:.1e}")
print("\n[Performance]")
print(f"  Training Time: {train_end_time - train_start_time:.2f} seconds")
print(f"  Total Script Time: {end_time - start_time:.2f} seconds")
print(f"  Final Total Loss: {losslist[-1]:.4e}")
print(f"  Relative L2 Error: {l2_error:.4e}")
print(f"  Relative H1 Error: {h1_error:.4e}")
print(f"  Relative H2 Error: {h2_error:.4e}")

Using device: cuda
Interior points: (20000, 2)
Boundary points: (10000, 2)
Forcing term: (20000, 1)
Dirichlet data: (10000, 1)
Clamped data: (10000, 1)
Starting training...
Epoch 0 | Total loss: 698.84240723 | Loss_int: 98.65694427 | Loss_Clamped: 0.03030297 | Loss_dirichlet: 0.02971558
Epoch 100 | Total loss: 0.58457196 | Loss_int: 0.40848932 | Loss_Clamped: 0.00000887 | Loss_dirichlet: 0.00000874
Epoch 200 | Total loss: 0.05389212 | Loss_int: 0.02966835 | Loss_Clamped: 0.00000106 | Loss_dirichlet: 0.00000136
Epoch 300 | Total loss: 0.04138168 | Loss_int: 0.01647637 | Loss_Clamped: 0.00000120 | Loss_dirichlet: 0.00000129
Epoch 400 | Total loss: 0.01848174 | Loss_int: 0.01105951 | Loss_Clamped: 0.00000023 | Loss_dirichlet: 0.00000051
Epoch 500 | Total loss: 0.25360596 | Loss_int: 0.01187196 | Loss_Clamped: 0.00001149 | Loss_dirichlet: 0.00001268
Epoch 600 | Total loss: 0.04028873 | Loss_int: 0.00726647 | Loss_Clamped: 0.00000197 | Loss_dirichlet: 0.00000133
Epoch 700 | Total loss: 0.80